In [1]:
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

path = '/home/joon/Documents/.font/NanumGothic.ttf'
font_name = fm.FontProperties(fname=path, size=14).get_name()
print(font_name)
plt.rc('font', family=font_name)

NanumGothic


In [2]:
!pip install contextualized-topic-models==2.2.0

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install pyldavis

Defaulting to user installation because normal site-packages is not writeable


In [21]:
text_file = "2016-10-20.txt"

In [22]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab
from tqdm import tqdm

In [23]:
documents = [line.strip() for line in open(text_file, encoding="utf-8").readlines()]

In [24]:
documents[:5]

['',
 '19  1990  52 1 22',
 '오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽

In [25]:
not '19  1990  52 1 22'.replace(' ', '').isdecimal()

False

In [26]:
preprocessed_documents = []

for line in tqdm(documents):
  # 빈 문자열이거나 숫자로만 이루어진 줄은 제외
  if line and not line.replace(' ', '').isdecimal():
    preprocessed_documents.append(line)


100%|██████████| 30091/30091 [00:00<00:00, 140677.51it/s]


In [43]:
print(preprocessed_documents[27001])

앵커  박근혜 대통령이 오늘 미르 스포츠 이 두 재단에 대해서 언급하자 검찰이 이제 좀 움직이는 것 같습니다 검찰은 두 재단관계자들에 대한 영장을 발부받아 통화 내역 추적에 들어갔습니다 실세로 의심받는 최순실씨는 독일로 출국한 것으로 보입니다  최우정 기자의 보도입니다  리포트  미르 스포츠 재단 관련 고발 사건을 수사 중인 검찰은 최근 최순실씨가 독일로 출국한 사실을 확인한 것으로 알려졌습니다  검찰은 최씨가 독일에서 생활해 온 딸 정유라 씨와 함께 있을 가능성이 높은 것으로 보고 있습니다  하지만 유럽연합 내 이동이 자유로운 만큼 다른 지역이나 미국 등지로 옮겼을 가능성도 배제하지 않고 있습니다  최씨의 위치를 파악한 뒤엔 직접 불러 조사할 가능성도 있는 것으로 알려졌습니다 검찰은 오늘 시민단체 고발장이 접수된 지 약 3주 만에 문체부 관계자들을 소환 조사했습니다  수사팀은 문체부 국장급 관계자 2명을 참고인으로 불러 미르재단 설립 경위 등을 물었습니다  검찰은 최근 수사검사를 2명에서 4명 이상으로 늘린 것으로 알려졌습니다  검찰은 사건을 특수부가 아닌 형사8부로 배당하고 수사팀 규모마저 작게 꾸려 미르 스포츠재단 수사의지가 없다는 비판을 받아 왔습니다  조선 최우정입니다  최우정 기자  뉴스제보 홈페이지 카카오톡 조선제보 전화 1661 0190  최우정 기자  조선 무단전재 및 재배포 금지


In [28]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [29]:
custom_tokenizer = CustomTokenizer(Mecab())

In [30]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [31]:
train_bow_embeddings = vectorizer.fit_transform(preprocessed_documents)

In [32]:
print(train_bow_embeddings.shape)

(27540, 3000)


In [16]:
vocab = vectorizer.get_feature_names()
id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}

In [17]:
len(vocab)

1013

In [18]:
train_contextualized_embeddings = bert_embeddings_from_list(preprocessed_documents, \
                                                            "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
qt = TopicModelDataPreparation()

training_dataset = qt.load(train_contextualized_embeddings, train_bow_embeddings, id2token)

In [20]:
ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, n_components=50, num_epochs=20)
ctm.fit(training_dataset)

0it [00:00, ?it/s]

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 50])

In [ ]:
ctm.get_topic_lists(10)

In [ ]:
ctm.get_topics(10)

In [ ]:
# 시각화
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)

In [ ]:
# 예측
import numpy as np

topics_predictions = ctm.get_thetas(training_dataset, n_samples=5)

In [ ]:
# 전처리 첫 번째 문서
print(preprocessed_documents[0])

In [ ]:
# 모델 저장 및 로드
ctm.save(models_dir="./")
del ctm

ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, num_epochs=100, n_components=50)
ctm.load("/content/contextualized_topic_model_nc_50_tpm_0.0_tpv_0.98_hs_prodLDA_ac_(100, 100)_do_softplus_lr_0.2_mo_0.002_rp_0.99",
                                                                                                      epoch=19)

In [ ]:
ctm.get_topics(5)